In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import unicodedata
import re
import os

In [19]:
from sentence_transformers import SentenceTransformer, util
from sklearn.neighbors import NearestNeighbors

# Initialize BERT model
model = SentenceTransformer('paraphrase-mpnet-base-v2')

## Load data

In [20]:
df_cra = pd.read_csv("../cra-data/cra_2021_combined_filtered.csv")
df_cra = df_cra.drop(columns=["Province", "Country"])
df_cra = df_cra.rename(columns={
    "BN": "CRA_BN_ID",
    "Category": "CRA_Category",
    "Sub Category": "CRA_SubCategory",
    "Designation": "CRA_Designation",
    "Legal Name": "CRA_LegalName",
    "Account Name": "Name",
    "Address Line 1": "CRA_Address1",
    "Address Line 2": "CRA_Address2",
    "City": "CRA_City",
    "Postal Code": "CRA_PostalCode",
    "Full Address": "CRA_FullAddress",
    "4050": "CRA_4050",
    "4155": "CRA_4155",
    "4850": "CRA_4850",
})

# Replace semicolons with " - " in all string columns
df_cra = df_cra.apply(lambda x: x.str.replace(';', ' - ') if x.dtype == 'object' else x)
# df_cra["Name"] = df_cra["Name"].str.split(pat=" - ", n=1).str[0]

df_cra = df_cra.drop_duplicates(subset=["Name"]).reset_index(drop=True)

In [21]:
# Clean and rename 211 data
df_211 = pd.read_csv("../211-data/2021_211_PeelYorkTO.csv", encoding='latin-1')
df_211 = df_211.drop(columns=["TaxonomyTerms"])
df_211 = df_211.rename(columns={
    "PublicName": "211_PublicName",
    "ParentAgency":"Name",
    "Address1":"211_Address1",
    "Address2":"211_Address2",
    "City":"211_City",
    "County":"211_County",
    "Province":"211_Province",
    "PostalCode":"211_PostalCode",
    "Latitude": "211_Latitude",
    "Longitude": "211_Longitude",
})

# Replace semicolons with " - " in all string columns
df_211 = df_211.apply(lambda x: x.str.replace(';', ' - ') if x.dtype == 'object' else x)

df_211 = df_211.drop_duplicates(subset=["211_PublicName", "211_Latitude", "211_Longitude"]).reset_index(drop=True)

In [23]:
df_cra

,CRA_BN_ID,CRA_Category,CRA_SubCategory,CRA_Designation,CRA_LegalName,Name,CRA_Address1,CRA_Address2,CRA_City,CRA_PostalCode,CRA_FullAddress,CRA_4050,CRA_4155,CRA_4850
0,770352110RR0001,1,15,C,Clarendon House Refugee Association,Clarendon House Refugee Association,6A - 2141 KIPLING AVE,NaN,ETOBICOKE,M9W4K8,"6A - 2141 KIPLING AVE,ETOBICOKE,ON",NaN,NaN,0.0
1,777522483RR0001,160,99,C,TR3:Programs,TR3:Programs,"215 FORT YORK BLVD, SUITE 2402",NaN,TORONTO,M5V4A2,"215 FORT YORK BLVD, SUITE 2402,TORONTO,ON",N,NaN,NaN
2,857642391RR0001,70,5,C,Alsalam Community Development Association,Alsalam Community Development Association,7-10 FALCONER DR,NaN,MISSISSAUGA,L5N3L8,"7-10 FALCONER DR,MISSISSAUGA,ON",NaN,50000.0,5399.0
3,717793939RR0001,30,99,B,Wisdom Core Ministries,Wisdom Core Ministries,132 MEADOW VISTA CRES,NaN,HOLLAND LANDING,L9N0T4,"132 MEADOW VISTA CRES,HOLLAND LANDING,ON",N,NaN,NaN
4,119159846RR0001,60,1,C,THE SPIRITUAL ASSEMBLY OF THE BAHA'IS OF WHITC...,SPIRITUAL ASSEMBLY OF THE BAHA'IS OF WHITCHURC...,93 FAIRVIEW AVENUE,NaN,STOUFFVILLE,L4A1C8,"93 FAIRVIEW AVENUE,STOUFFVILLE,ON",N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6514,822347514RR0001,50,3,C,CHABAD JEWISH STUDENT CENTRE,Chabad Jewish Student Centre,6 MILNER GATE,NaN,THORNHILL,L4J2N4,"6 MILNER GATE,THORNHILL,ON",NaN,873301.0,22848.0
6515,817013873RR0001,30,11,C,PRAISE THE LORD CHURCH,PRAISE THE LORD CHURCH,5765 YONGE ST. #101,NaN,TORONTO,M2M4H9,"5765 YONGE ST. #101,TORONTO,ON",NaN,NaN,NaN
6516,118987494RR0001,1,99,C,KUPATH RAMBAN CANADA,KUPATH RAMBAN CANADA,14 BRETT AVE,NaN,TORONTO,M3H2W6,"14 BRETT AVE,TORONTO,ON",N,NaN,NaN
6517,132410671RR0266,1,99,C,Sociï¿½tï¿½ de Saint-Vincent de Paul-Conseil n...,CONFERENCE ST-LOUIS DE FRANCE,1415 DON MILLS ROAD,NaN,NORTH YORK,M3B2X5,"1415 DON MILLS ROAD,NORTH YORK,ON",N,NaN,NaN


In [24]:
df_211

,211_PublicName,Name,211_Address1,211_Address2,211_City,211_County,211_Province,211_PostalCode,211_Latitude,211_Longitude
0,Doane House Hospice - Doane House Hospice,Doane House Hospice,17100 Yonge St,NaN,Newmarket,York,ON,L3Y 8V3,44.045718,-79.478804
1,Markham Board of Trade - Markham Board of Trade,Markham Board of Trade,3600 Steeles Ave E,C1 - Suite 105,Markham,York,ON,L3R 9Z7,43.818293,-79.334058
2,Bell Media - Bell Media,Bell Media,299 Queen St W,NaN,Toronto,Toronto,ON,M5V 2Z5,43.649840,-79.390509
3,York Region. Community & Health Services (Inte...,York Region. Community & Health Services (Inte...,17250 Yonge Street,NaN,Newmarket,York,ON,L3Y 6Z1,44.048254,-79.480218
4,York Region. Community & Health Services (Publ...,York Region. Community & Health Services (Publ...,50 High Tech Road,NaN,Richmond Hill,York,ON,L4B 4N7,43.842851,-79.427961
...,...,...,...,...,...,...,...,...,...,...
6400,"Dynacare - COVID-19 Assessment Centre, Dynacar...",Dynacare,"7700 Hurontario St, Unit 101",NaN,Brampton,Peel,ON,L6Y 4M3,43.659060,-79.723910
6401,"Dynacare - COVID-19 Assessment Centre, Dynacar...",Dynacare,"10 Cottrelle Blvd, Unit 108",NaN,Brampton,Peel,ON,L6S 0E2,43.761280,-79.712790
6402,"Dynacare - COVID-19 Assessment Centre, Dynacar...",Dynacare,"9525 Mississauga Rd, Units 8 and 9",NaN,Brampton,Peel,ON,L6X 0Z8,43.658170,-79.817190
6403,Holland Bloorview Kids Rehabilitation Hospital...,Holland Bloorview Kids Rehabilitation Hospital,150 Kilgour Rd,NaN,Toronto,Toronto,ON,M4G 1R8,43.718295,-79.374274


## Algorithmic matching

In [27]:
# Clean text functions
def clean_text(text):
    if pd.isna(text):
        return ''
    text = unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'[^a-zA-Z0-9]', '', text).lower()
    return text

def clean_text_with_spaces(text):
    if pd.isna(text):
        return ''
    text = unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('ascii')
    # Keep spaces but remove other special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text).lower()
    # Collapse multiple spaces into one
    text = re.sub(r' +', ' ', text).strip()
    return text

# Prepare CRA data
df_cra_prep = df_cra.copy()
df_cra_prep['clean_name'] = df_cra['Name'].apply(clean_text)
df_cra_prep['clean_name_space'] = df_cra['Name'].apply(clean_text_with_spaces)
df_cra_prep['PostalCode_clean'] = df_cra['Postal Code'].str.replace(r'\s+', '', regex=True)

# Prepare 211 data
df_211_prep = df_211.copy()
df_211_prep['clean_name'] = df_211['Name'].apply(clean_text)
df_211_prep['clean_name_space'] = df_211['Name'].apply(clean_text_with_spaces)
df_211_prep['PostalCode_clean'] = df_211['211_PostalCode'].str.replace(r'\s+', '', regex=True)

print("Data preparation complete")
print(f"CRA records: {len(df_cra_prep)}")
print(f"211 records: {len(df_211_prep)}")

Data preparation complete
CRA records: 6519
211 records: 6405


In [28]:
# STEP 1: Input manual matches we've already found

df_manual_map = pd.read_csv('../joined-data/manual_matches.csv')

# Merge with df_cra_prep on CRA_name <-> Name
df_cra_joined = df_manual_map.merge(df_cra_prep, left_on='CRA_name', right_on='Name', how='left', suffixes=('', '_cra'))

# Merge with df_211_prep on 211_name <-> Name
df_full_joined = df_cra_joined.merge(df_211_prep, left_on='211_name', right_on='Name', how='left', suffixes=('', '_211'))

# df_full_joined.to_csv('../joined-data/manual_matches_joined.csv', index=False)

remaining_cra = df_cra_prep[~df_cra_prep['clean_name'].isin(df_full_joined['clean_name'].unique())]
remaining_211 = df_211_prep[~df_211_prep['clean_name'].isin(df_full_joined['clean_name_211'].unique())]

print(f"Step 1 Complete: {len(df_full_joined)} matched rows")
print(f"Unique CRA orgs matched: {df_full_joined['BN'].nunique()}")
print(f"Unique 211 orgs matched: {df_full_joined['clean_name_211'].nunique()}")
print(f"Remaining CRA records: {len(remaining_cra)}")
print(f"Remaining 211 branches: {len(remaining_211)}")

Step 0 Complete: 288 matched rows
Unique CRA orgs matched: 94
Unique 211 orgs matched: 94
Remaining CRA records: 6425
Remaining 211 branches: 6117


In [6]:
# STEP 2: Exact name matching
exact_matches = pd.merge(
    df_cra_prep,
    df_211_prep,
    on='clean_name',
    how='inner',
    suffixes=('_cra', '_211')
)

if not exact_matches.empty:
    matched_orgs = exact_matches['clean_name'].unique()
    all_branches = df_211_prep[df_211_prep['clean_name'].isin(matched_orgs)]
    
    exact_match_results = pd.merge(
        df_cra_prep[df_cra_prep['clean_name'].isin(matched_orgs)],
        all_branches,
        on='clean_name',
        how='left',
        suffixes=('_cra', '_211')
    )
    exact_match_results['Match_Method'] = 'Exact Name'
else:
    exact_match_results = pd.DataFrame()

remaining_cra = df_cra_prep[~df_cra_prep['clean_name'].isin(exact_match_results['clean_name'].unique())]
remaining_211 = df_211_prep[~df_211_prep['clean_name'].isin(exact_match_results['clean_name'].unique())]

print(f"Step 1 Complete: {len(exact_match_results)} matched rows")
print(f"Unique CRA orgs matched: {exact_match_results['BN'].nunique()}")
print(f"Unique 211 orgs matched: {exact_match_results['clean_name'].nunique()}")
print(f"Remaining CRA records: {len(remaining_cra)}")
print(f"Remaining 211 branches: {len(remaining_211)}")

Step 1 Complete: 1094 matched rows
Unique CRA orgs matched: 393
Unique 211 orgs matched: 393
Remaining CRA records: 6126
Remaining 211 branches: 5311


In [7]:
# STEP 3: Postal code + fuzzy name matching using clean_name_space
common_postal_codes = set(remaining_cra['PostalCode_clean']).intersection(set(remaining_211['PostalCode_clean']))
postal_matched_results = []
postal_bad_matches = []

for postal_code in tqdm(common_postal_codes):
    cra_subset = remaining_cra[remaining_cra['PostalCode_clean'] == postal_code]
    two11_subset = remaining_211[remaining_211['PostalCode_clean'] == postal_code]
    
    if len(cra_subset) == 0 or len(two11_subset) == 0:
        continue
        
    cra_embeddings = model.encode(cra_subset['clean_name_space'].tolist())
    two11_embeddings = model.encode(two11_subset['clean_name_space'].tolist())
    
    nn = NearestNeighbors(n_neighbors=1, metric='cosine').fit(two11_embeddings)
    distances, indices = nn.kneighbors(cra_embeddings, return_distance=True)
    
    for i, (distance, index) in enumerate(zip(distances, indices)):
        similarity = 1 - distance[0]
        if similarity >= 0.80:
            matched_org_name = two11_subset.iloc[index[0]]['clean_name_space']
            matched_branches = remaining_211[remaining_211['clean_name_space'] == matched_org_name]
            
            for _, branch_row in matched_branches.iterrows():
                merged_row = cra_subset.iloc[i].to_dict()
                merged_row.update({f'{k}_211': v for k, v in branch_row.to_dict().items()})
                merged_row['Match_Method'] = 'Postal Code + Fuzzy Name'
                merged_row['Similarity_Score'] = similarity
                postal_matched_results.append(merged_row)
        else:
            bad_row = {
                'CRA_Name': cra_subset.iloc[i]['Name'],
                '211_Name': two11_subset.iloc[index[0]]['Name'],
                'Postal_Code': postal_code,
                'Similarity_Score': similarity,
                'Match_Status': 'Below Threshold'
            }
            postal_bad_matches.append(bad_row)

postal_fuzzy_results = pd.DataFrame(postal_matched_results) if postal_matched_results else pd.DataFrame()
postal_bad_matches_df = pd.DataFrame(postal_bad_matches) if postal_bad_matches else pd.DataFrame()

remaining_cra = remaining_cra[~remaining_cra['BN'].isin(postal_fuzzy_results['BN'])]
remaining_211 = remaining_211[~remaining_211['clean_name_space'].isin(postal_fuzzy_results['clean_name_space_211'])]

print(f"\nStep 3 Complete: {len(postal_fuzzy_results)} matched rows")
print(f"Unique CRA orgs matched: {postal_fuzzy_results['BN'].nunique()}")
print(f"Unique 211 orgs matched: {postal_fuzzy_results['clean_name_space_211'].nunique()}")
print(f"Remaining CRA records: {len(remaining_cra)}")
print(f"Remaining 211 branches: {len(remaining_211)}")

100%|█████████████████████████████████████████| 852/852 [01:53<00:00,  7.50it/s]


Step 2 Complete: 504 matched rows
Unique CRA orgs matched: 209
Unique 211 orgs matched: 206
Remaining CRA records: 5917
Remaining 211 branches: 4840


In [8]:
# STEP 4: General fuzzy matching using clean_name_space
general_matched_results = []
general_bad_matches = []

cra_embeddings = model.encode(remaining_cra['clean_name_space'].tolist())
two11_embeddings = model.encode(remaining_211['clean_name_space'].tolist())
print('Embeddings generated')

nn = NearestNeighbors(n_neighbors=1, metric='cosine').fit(two11_embeddings)
distances, indices = nn.kneighbors(cra_embeddings, return_distance=True)
print('Nearest neighbours computed')

for i in tqdm(range(len(remaining_cra))):
    similarity = 1 - distances[i][0]
    if similarity >= 0.80:
        matched_org_name = remaining_211.iloc[indices[i][0]]['clean_name_space']
        matched_branches = remaining_211[remaining_211['clean_name_space'] == matched_org_name]
        
        for _, branch_row in matched_branches.iterrows():
            merged_row = remaining_cra.iloc[i].to_dict()
            merged_row.update({f'{k}_211': v for k, v in branch_row.to_dict().items()})
            merged_row['Match_Method'] = 'General Fuzzy Match'
            merged_row['Similarity_Score'] = similarity
            general_matched_results.append(merged_row)
    else:
        bad_row = {
            'CRA_Name': remaining_cra.iloc[i]['Name'],
            '211_Name': remaining_211.iloc[indices[i][0]]['Name'],
            'Similarity_Score': similarity,
            'Match_Status': 'Below Threshold'
        }
        general_bad_matches.append(bad_row)

general_fuzzy_results = pd.DataFrame(general_matched_results) if general_matched_results else pd.DataFrame()
general_bad_matches_df = pd.DataFrame(general_bad_matches) if general_bad_matches else pd.DataFrame()

remaining_cra = remaining_cra[~remaining_cra['BN'].isin(general_fuzzy_results['BN'])]
remaining_211 = remaining_211[~remaining_211['clean_name_space'].isin(general_fuzzy_results['clean_name_space_211'])]

print(f"\nStep 4 Complete: {len(general_fuzzy_results)} matched rows")
print(f"Unique CRA orgs matched: {general_fuzzy_results['BN'].nunique()}")
print(f"Unique 211 orgs matched: {general_fuzzy_results['clean_name_space_211'].nunique()}")
print(f"Final Unmatched CRA records: {len(remaining_cra)}")
print(f"Final Unmatched 211 branches: {len(remaining_211)}")

Embeddings generated
Nearest neighbours computed


100%|█████████████████████████████████████| 5917/5917 [00:00<00:00, 6278.98it/s]


Step 3 Complete: 461 matched rows
Unique CRA orgs matched: 245
Unique 211 orgs matched: 122
Final Unmatched CRA records: 5672
Final Unmatched 211 branches: 4538


## Clean-up, combine, save

In [9]:
exact_match_updated = exact_match_results.drop(
    columns=['clean_name', 'clean_name_space_cra', 'clean_name_space_211', 'PostalCode_clean_cra', 'PostalCode_clean_211']
).rename(
    columns={
        'PublicName': '211_PublicName',
        'Name_211': '211_Name',
        'Latitude': '211_Latitude', 
        'Longitude': '211_Longitude',
    }
)

postal_fuzzy_updated = postal_fuzzy_results.drop(
    columns=['clean_name', 'clean_name_space', 'clean_name_211', 'clean_name_space_211', 'PostalCode_clean', 'PostalCode_clean_211']
).rename(
    columns={
        'Name': 'Name_cra',
        'PublicName_211': '211_PublicName',
        'Name_211': '211_Name',
        '211_Address1_211': '211_Address1', 
        '211_Address2_211': '211_Address2',
        '211_City_211': '211_City', 
        '211_County_211': '211_County', 
        '211_Province_211': '211_Province',
        '211_PostalCode_211': '211_PostalCode', 
        'Latitude_211': '211_Latitude', 
        'Longitude_211': '211_Longitude',
    }
)

general_fuzzy_updated = general_fuzzy_results.drop(
    columns=['clean_name', 'clean_name_space', 'clean_name_211', 'clean_name_space_211', 'PostalCode_clean', 'PostalCode_clean_211']
).rename(
    columns={
        'Name': 'Name_cra',
        'PublicName_211': '211_PublicName',
        'Name_211': '211_Name',
        '211_Address1_211': '211_Address1', 
        '211_Address2_211': '211_Address2',
        '211_City_211': '211_City', 
        '211_County_211': '211_County', 
        '211_Province_211': '211_Province',
        '211_PostalCode_211': '211_PostalCode', 
        'Latitude_211': '211_Latitude', 
        'Longitude_211': '211_Longitude',
    }
)

In [30]:
# Manually identified mismatches for removal
bad_orgs_211 = [
    "Toronto Catholic District School Board",
    "York Region Children's Aid Society",
    "Toronto Mental Health and Addictions Access Point",
    "Toronto Public Health",
    "Canadian Cultural Society of the Deaf",
    "Canadian Environmental Law Association",
    "Friends of Jesus Christ",
    "Toronto. Shelter, Support and Housing Administration. Homelessness Initiatives and Prevention Services",
    "Salvation Army of Georgina Community Church",
    "Matthew House Refugee Reception Services, Toronto",
    "Salvation Army of Georgina Community Church",
    "International Schizophrenia Foundation",
    "Ontario Addiction Treatment Centres",
    "St Peter's Seniors' Residence",
    "Ukrainian Canadian Social Services (Toronto)",
    "Baycrest Child Care Centre",
    "Stewardship Ontario",
    "Canada. Health Canada",
    "Brampton Life Centre",
    "Chabad Lubavitch Of York Mills",
    "Registered Nurses' Association of Ontario",
    "Ontario. Ministry of Children, Community and Social Services",
    "Canadian Cultural Society of the Deaf",
    "Redeemed Christian Church of God",
    "St. James the Apostle Anglican Church",
    "Redeemed Christian Church of God",
    "Royal Day Care Centre",
    "Peel Region",
    "Society of St Vincent de Paul.[Newmarket / East Gwillimbury Region]",
    "Cypriot Community of Toronto",
    "Seventh-Day Adventist Church",
    "Ontario Association on Developmental Disabilities (The)",
    "Eye Bank of Canada",
    "Canadian Training Institute",
    "Maple Hill Baptist Church",
    "Friends of Jesus Christ",
    "Epilepsy South Central Ontario",
    "Toronto. Children's Services",
    "Toronto Community Housing",
    "Unionville Montessori School Church",
    "My House",
    "Chabad Lubavitch Of York Mills",
    "Canada. Health Canada",
    "Arab Community Centre of Toronto",
    "Toronto. Employment and Social Services",
    "Arab Community Centre of Toronto",
    "Mississauga, City of",
    "Chinese Canadian National Council, Toronto Chapter",
    "Ontario. Ministry of Labour, Training and Skills Development. Employment Practices Branch",
    "St Bartholomew's Anglican Church, Regent Park Community Services",
    "Vietnamese Community Centre of Mississauga",
    "Somali Canadian Association of Etobicoke",
    "Willowdale Manor",
    "Father Henri Nouwen",
    "Registered Nurses' Association of Ontario",
    "Epilepsy South Central Ontario",
    "Bereavement Authority of Ontario",
    "Autism in Mind",
    "North American Muslim Foundation",
    "Lighthouse (The)",
    "Our Saviour Lutheran Church Food Bank",
    "National Council of Jewish Women of Canada",
    "Open Arms Preschool",
    "VIVA Thornhill Woods",
    "Chabad Lubavitch Of York Mills",
    "Children's Aid Foundation of Canada",
    "Women's Health Matters",
    "Childhood Cancer Canada",
    "Somali Canadian Association of Etobicoke",
    "Mississauga Food Bank",
    "Armenian Relief Society Toronto Roubina Chapter",
]

In [41]:
# Combine all successful matches
all_matches = pd.concat([
    exact_match_updated,
    postal_fuzzy_updated,
    general_fuzzy_updated
], ignore_index=True)

incorrect_matches = all_matches[all_matches['211_Name'].isin(bad_orgs_211)]
incorrect_matches.loc[:, 'Match_Method'] = incorrect_matches['Match_Method'] + " - incorrect"
all_matches = all_matches[~all_matches['211_Name'].isin(bad_orgs_211)]

In [42]:
# Create output directory
os.makedirs('../joined-data', exist_ok=True)

# Create non-faith matches
remaining_cra_nonfaith = remaining_cra[~((remaining_cra['Category'] >= 30) & (remaining_cra['Category'] <= 90))] 

# Save all results
all_matches.to_csv('../joined-data/successful_matches.csv', index=False)
remaining_cra.to_csv('../joined-data/unmatched_cra_records.csv', index=False)
remaining_cra_nonfaith.to_csv('../joined-data/unmatched_cra_nonfaith_records.csv', index=False)
remaining_211.to_csv('../joined-data/unmatched_211_records.csv', index=False)
pd.concat([postal_bad_matches_df, general_bad_matches_df, incorrect_matches]).to_csv('../joined-data/failed_fuzzy_matches.csv', index=False)

# Create simplified version
simplified_matches = all_matches[[
    'BN', 'Name_cra', '211_Name', '211_PublicName', 
    '211_Address1', '211_PostalCode', '211_Longitude', '211_Latitude', 'Match_Method'
]].rename(columns={
    'BN': 'CRA_BN_ID',
    'Name_cra': 'CRA_Organization_Name',
    '211_Name': '211_Organization_Name',
    '211_PublicName': '211_Location_Name',
    '211_Address1': '211_Address',
    '211_PostalCode': '211_Postal_Code',
    '211_Longitude': 'X_Coordinate',
    '211_Latitude': 'Y_Coordinate'
})

simplified_matches.to_csv('../joined-data/simplified_matches.csv', index=False)

print("\nFinal Results:")
print(f"- Successful matches: {len(all_matches)} records")
print(f"- Matched CRA organizations: {all_matches['Name_cra'].nunique()} records")
print(f"- Unmatched CRA organizations: {len(remaining_cra)} records")
print(f"- Unmatched CRA (non-faith) organizations: {len(remaining_cra_nonfaith)} records")
print(f"- Matched 211 branches: {all_matches['211_PublicName'].nunique()} records")
print(f"- Unmatched 211 branches: {len(remaining_211)} records")
print(f"- Simplified matches saved with {len(simplified_matches)} records")


Final Results:
- Successful matches: 1747 records
- Matched CRA organizations: 668 records
- Unmatched CRA organizations: 5672 records
- Unmatched CRA (non-faith) organizations: 1308 records
- Matched 211 branches: 1680 records
- Unmatched 211 branches: 4538 records
- Simplified matches saved with 1747 records
